In [ ]:
import pandas as pd

# Definiamo i nomi delle colonne
column_names = [
    'id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean',
    'smoothness_mean', 'compactness_mean', 'concavity_mean', 'concave_points_mean',
    'symmetry_mean', 'fractal_dimension_mean', 'radius_se', 'texture_se',
    'perimeter_se', 'area_se', 'smoothness_se', 'compactness_se', 'concavity_se',
    'concave_points_se', 'symmetry_se', 'fractal_dimension_se', 'radius_worst',
    'texture_worst', 'perimeter_worst', 'area_worst', 'smoothness_worst',
    'compactness_worst', 'concavity_worst', 'concave_points_worst', 'symmetry_worst',
    'fractal_dimension_worst'
]

data = pd.read_csv('wdbc.data', header=None, names=column_names)

In [ ]:
from imblearn.over_sampling import SMOTE

# Separiamo le features dal target
X = data.drop(columns=['id', 'diagnosis'])
y = data['diagnosis']

# Applichiamo SMOTE per bilanciare il dataset
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Creiamo un nuovo DataFrame bilanciato
data_resampled = pd.concat([pd.DataFrame(X_resampled, columns=X.columns), pd.Series(y_resampled, name='diagnosis')], axis=1)

print("Distribuzione delle classi prima di SMOTE:")
print(y.value_counts())
print("\nDistribuzione delle classi dopo SMOTE:")
print(y_resampled.value_counts())

# Sovrascriviamo il DataFrame originale con quello bilanciato per le celle successive
data = data_resampled

In [ ]:
from pgmpy.estimators import HillClimbSearch, K2Score
from pgmpy.models import BayesianNetwork
from pgmpy.inference import VariableElimination
from sklearn.preprocessing import KBinsDiscretizer

features = data.drop(columns=['diagnosis'])
target = data['diagnosis']

discretizer = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='uniform', subsample=None)
features_disc = pd.DataFrame(discretizer.fit_transform(features), columns=features.columns, index=data.index).astype(int)

data_disc = pd.concat([features_disc, target], axis=1)

In [ ]:
scoring_method = K2Score(data_disc)
hc = HillClimbSearch(data_disc)
best_model = hc.estimate(scoring_method=scoring_method, max_indegree=5, max_iter=int(1e4))
print("Archi appresi:", best_model.edges())

bn = BayesianNetwork(best_model.edges())
bn.fit(data_disc)

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

plt.figure(figsize=(40, 40))

G = nx.DiGraph(best_model.edges())

node_colors = ['red' if node == 'diagnosis' else 'skyblue' for node in G.nodes()]

pos = nx.spring_layout(G, k=2.5, iterations=150)
nx.draw(G, pos, with_labels=True, node_size=5000, node_color=node_colors, font_size=25, font_weight='bold', arrows=True, arrowsize=23)
plt.title("Struttura della Rete Bayesiana Appresa", size=23)
plt.show()

In [ ]:
markov_blanket = bn.get_markov_blanket('diagnosis')

print(f"La Markov Blanket di 'diagnosis' è: {markov_blanket}")
print(f"\nNumero di features nella Markov Blanket: {len(markov_blanket)}")
print(f"\nSono state eliminate {32 - len(markov_blanket)} features.")
print("\nQueste sono le uniche variabili necessarie per predire la diagnosi secondo il modello appreso.")
print("Le altre variabili possono essere considerate ridondanti per questo scopo.")

features_ridotte = markov_blanket.copy()
if 'diagnosis' not in features_ridotte:
    features_ridotte.append('diagnosis')

data_ridotto = data[features_ridotte]

In [ ]:
G_ridotto = G.subgraph(features_ridotte)

plt.figure(figsize=(20, 20))

node_colors_ridotto = ['red' if node == 'diagnosis' else 'skyblue' for node in G_ridotto.nodes()]

pos_ridotto = nx.spring_layout(G_ridotto, k=3.5, iterations=150)
nx.draw(G_ridotto, pos_ridotto, with_labels=True, node_size=6000, node_color=node_colors_ridotto, font_size=20, font_weight='bold', arrows=True, arrowsize=20)
plt.title("Sottografo della Rete Bayesiana: (Markov Blanket)", size=25)
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import seaborn as sns

X_ridotto = data_ridotto.drop('diagnosis', axis=1)
y_ridotto_str = data_ridotto['diagnosis']

X_ridotto_disc = pd.DataFrame(discretizer.fit_transform(X_ridotto), columns=X_ridotto.columns, index=X_ridotto.index).astype(int)

le = LabelEncoder()
y_ridotto = pd.Series(le.fit_transform(y_ridotto_str), name='diagnosis', index=y_ridotto_str.index)

data_ridotto_disc = pd.concat([X_ridotto_disc, y_ridotto], axis=1)

X_train, X_test, y_train, y_test = train_test_split(
    X_ridotto_disc, y_ridotto, test_size=0.3, random_state=42, stratify=y_ridotto
)

bn_ridotto = BayesianNetwork(G_ridotto.edges())
bn_ridotto.fit(pd.concat([X_train, y_train], axis=1))

inference = VariableElimination(bn_ridotto)
y_pred = []

X_test_for_pred = X_test.copy()

for index, row in X_test_for_pred.iterrows():
    pred = inference.map_query(variables=['diagnosis'], evidence=row.to_dict())
    y_pred.append(pred['diagnosis'])

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, target_names=le.classes_, output_dict=True)
report_str = classification_report(y_test, y_pred, target_names=le.classes_)

print(f"Accuracy del modello ridotto: {accuracy:.4f}\n")
print("Report di Classificazione del Modello Ridotto:\n")
print(report_str)


In [ ]:
random_samples = bn_ridotto.simulate(n_samples=400, seed=42)
diagnosis_counts = random_samples['diagnosis'].value_counts()
print("Numero di campioni generati per ciascuna diagnosi:\n", diagnosis_counts)
print(random_samples)


In [ ]:
import seaborn as sns

import matplotlib.pyplot as plt

# Ordiniamo i conteggi per indice per garantire che le etichette corrispondano (B, M)
ordered_counts = diagnosis_counts.sort_index()

# Otteniamo le etichette delle classi ('B', 'M') dal LabelEncoder
class_labels = le.inverse_transform(ordered_counts.index)

# Creiamo il grafico a barre
plt.figure(figsize=(8, 6))
barplot = sns.barplot(x=class_labels, y=ordered_counts.values, palette="viridis")

# Aggiungiamo titolo ed etichette agli assi
plt.title("Distribuzione dei Campioni Sintetici Generati", fontsize=16)
plt.xlabel("Diagnosi", fontsize=12)
plt.ylabel("Numero di Campioni", fontsize=12)

# Aggiungiamo i valori sopra le barre
for i in barplot.containers:
    barplot.bar_label(i,)

plt.show()

# Creiamo il grafico a torta per mostrare le percentuali
plt.figure(figsize=(8, 8))
plt.pie(ordered_counts.values, labels=class_labels, autopct='%1.1f%%', startangle=140, colors=['lightcoral', 'lightskyblue'])
plt.title("Percentuale dei Campioni Sintetici per Diagnosi", fontsize=16)
plt.axis('equal')  # Assicura che il grafico a torta sia disegnato come un cerchio.
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Estraiamo le medie delle metriche dal report
avg_precision = report['macro avg']['precision']
avg_recall = report['macro avg']['recall']
avg_f1 = report['macro avg']['f1-score']

metric_labels = ['Precision', 'Recall', 'F1-Score']
metric_values = [avg_precision, avg_recall, avg_f1]

fig, ax = plt.subplots(figsize=(10, 7))
bars = ax.bar(metric_labels, metric_values, color=['skyblue', 'lightgreen', 'salmon'])

ax.set_ylabel('Punteggio')
ax.set_title('Metriche Medie di Valutazione del Modello Ridotto')
ax.set_ylim(0, 1.1)

# Aggiungiamo la linea dell'accuracy per confronto
ax.axhline(y=accuracy, color='r', linestyle='--', label=f'Accuracy: {accuracy:.3f}')
ax.legend()

ax.bar_label(bars, padding=3, fmt='%.3f')

fig.tight_layout()
plt.show()